<a href="https://colab.research.google.com/github/PiyumaliSandunika/e18-4yp-Multimodal-Emotion-Prediction-Using-Reinforcement-Learning/blob/main/Text_model/online_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


** Model Loading**

In [2]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Load the saved model
model_path = "/content/drive/MyDrive/FYP_Text/"
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
def predict_emotion_with_probabilities(sentence, model, tokenizer):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Convert probabilities tensor to list
    probabilities_list = probabilities.squeeze().tolist()

    # Map predicted label to emotion
    emotion_labels = ["happiness", "sadness", "anger", "neutral"]
    predicted_emotion = emotion_labels[torch.argmax(logits, dim=1).item()]

    return probabilities_list, predicted_emotion


In [ ]:

# Example sentences for prediction
sentences = [
    # "I am a broken-hearted",
    # "I am emotionaly damaged right now",
    # "Do what makes feel you better",
    # "I feel nervous",
    # "I am a student",
    # "You're stupid and it makes me anger",
    # "Go to hell",
    # "You are the worst,I do not want to see you",
    # "I am angry right now",
    # "You are my sunshine",
    # "Such a wonderful performance",
    # "I am lonely",
    # "This is strange! anyways bravo to myself!",
    # "Things are getting stranger",
    # "I lumped my throat",
    # "Billy, your face is like a wet weekend",
    "I’m on cloud nine"
]

# Perform prediction for each sentence
for sentence in sentences:
    predicted_emotion = predict_emotion_with_probabilities(sentence,loaded_model,tokenizer)
    print(f"Sentence: '{sentence}'")
    print(f"Predicted Emotion: {predicted_emotion}\n")

Sentence: 'I’m on cloud nine'
Predicted Emotion: ([2.455699586789706e-06, 2.885532694563153e-06, 1.1664121757348767e-06, 0.9999934434890747], 'neutral')



In [ ]:
def active_learning_selection(user_data, model, threshold=0.5):
    selected_examples = []
    for text, label in user_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**encoded_input)
        probabilities = torch.softmax(outputs.logits, dim=1)
        confidence = probabilities.max().item()
        if confidence < threshold:
            selected_examples.append((text, label))
    return selected_examples

# Function for online learning update
def online_learning_update(new_data, model, optimizer, loss_fn):
    for text, label in new_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**encoded_input, labels=torch.tensor([label]))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return model
def is_confident(predicted_label, confidence_threshold=0.7):
    # Check if the prediction confidence is above the threshold
    return confidence_threshold <= torch.softmax(predicted_label, dim=1).max()

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Initialize DistilBERT tokenizer and load the fine-tuned model
model_path = "/content/drive/MyDrive/FYP_Text/"
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define label mappings
label_map = {
    0: "happiness",
    1: "sadness",
    2: "anger",
    3: "neutral"
}

# Function for online learning update
def online_learning_update(new_data, model, optimizer, loss_fn):
    for text, label in new_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor([label])
        outputs = model(**encoded_input)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return model

# Main loop for user interaction
user_data = []

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Optimizer for online learning
loss_fn = torch.nn.CrossEntropyLoss()  # Loss function for online learning

while True:
    # Receive input from the user
    user_input = input("Enter text: ")

    # Perform inference with the current model
    encoded_input = tokenizer(user_input, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**encoded_input)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(f"Predicted emotion: {label_map[predicted_label]}")

    # Ask for user feedback
    user_label = int(input("Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): "))
    user_data.append((user_input, user_label))

    # Retrain the model with the new data
    model = online_learning_update([(user_input, user_label)], model, optimizer, loss_fn)

    # Perform inference with the updated model
    outputs = model(**encoded_input)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(f"New predicted emotion: {label_map[predicted_label]}")

    # Prompt the user for more input
    more_data = input("Do you have more data to provide? (yes/no): ")
    if more_data.lower() != "yes":
        break


Enter text: I’m on cloud nine
Predicted emotion: neutral
Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): 0
New predicted emotion: happiness
Do you have more data to provide? (yes/no): no


In [ ]:
# Perform inference with the updated model
encoded_input = tokenizer("I'm on cloud", padding=True, truncation=True, return_tensors='pt')
outputs = model(**encoded_input)
predicted_label = torch.argmax(outputs.logits, dim=1).item()
print(f"New predicted emotion: {label_map[predicted_label]}")


Evaluation with Holdout data

In [22]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd

# Initialize DistilBERT tokenizer and load the fine-tuned model
model_path = "/content/drive/MyDrive/FYP_Text/"
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define label mappings
label_map = {
    0: "happiness",
    1: "sadness",
    2: "anger",
    3: "neutral"
}

# Function for online learning update
def online_learning_update(new_data, model, optimizer, loss_fn):
    for text, label in new_data:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor([label])
        outputs = model(**encoded_input)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return model

# Function to evaluate model
def evaluate_model(model, tokenizer, holdout_data):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for text, label in holdout_data:
            encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
            labels = torch.tensor([label])
            outputs = model(**encoded_input, labels=labels)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Main loop for user interaction
user_data = []

optimizer = torch.optim.AdamW(loaded_model.parameters(), lr=5e-5)  # Optimizer for online learning
loss_fn = torch.nn.CrossEntropyLoss()  # Loss function for online learning

# Holdout data for evaluation
# holdout_data = [
#     ("text1", 0),  # Example data with label
#     ("text2", 1),
#     # Add more holdout data as needed
# ]



In [26]:
# Load holdout data from CSV
holdout_data_file = "dataset2.csv"
holdout_df = pd.read_csv(holdout_data_file)
# Shuffle the DataFrame
holdout_df = holdout_df.sample(frac=1).reset_index(drop=True)
# Extract sentences and labels
texts = holdout_df['content'].tolist()
holdout_df["emotion"] = holdout_df["emotion"].str.strip()
labels = holdout_df['emotion'].tolist()
# Convert emotion labels to numerical values
label_map2 = {"happiness": 0, "sadness": 1, "anger": 2, "neutral": 3}
labels = [label_map2[label] for label in labels]
# Convert holdout data to list of tuples
holdout_data = list(zip(texts,labels))

[('Discovered a dope bookstore', 0), ("I'm so angry I could punch something", 2), ('Doing homework in my room', 3), ('Cleaning my room', 3), ('Got a handwritten note from crush', 0), ("Playing with a Rubik's cube", 3), ('Feeling misunderstood', 1), ('Practicing a musical instrument', 3), ('Trying out a new recipe', 3), ('Going for a walk outside', 3), ('Planning my weekend with friends', 3), ("I'm so frustrated right now", 2), ('Lost in a captivating book', 0), ("Can't believe you would do that", 2), ('Deep grief settling in', 1), ("I'm so mad I can't even think straight", 2), ('Hanging out at the mall', 3), ('Surprised with flowers', 0), ('Doing some online shopping', 3), ('Completed a creative project', 0), ("Feeling like I'm about to explode", 2), ('Checking social media', 3), ('Nature walk vibes', 0), ("I can't deal with this anymore", 2), ('Dance party vibes', 0), ("I can't deal with your incompetence", 2), ('Dreams slipping away', 1), ('Seriously annoyed right now', 2), ('Left ou

In [31]:

# Initial evaluation
initial_accuracy = evaluate_model(loaded_model, tokenizer, holdout_data)
print("Initial accuracy:", initial_accuracy)

while True:
    # Receive input from the user
    user_input = input("Enter text: ")

    # Perform inference with the current model
    encoded_input = tokenizer(user_input, padding=True, truncation=True, return_tensors='pt')
    outputs = loaded_model(**encoded_input)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(f"Predicted emotion: {label_map[predicted_label]}")

    # Ask for user feedback
    user_label = int(input("Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): "))
    user_data.append((user_input, user_label))

    # Retrain the model with the new data
    loaded_model = online_learning_update([(user_input, user_label)], loaded_model, optimizer, loss_fn)

    # Perform inference with the updated model
    outputs = loaded_model(**encoded_input)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(f"New predicted emotion: {label_map[predicted_label]}")

    # Prompt the user for more input
    more_data = input("Do you have more data to provide? (yes/no): ")
    if more_data.lower() != "yes":
        break

# Evaluate model after update
updated_accuracy = evaluate_model(loaded_model, tokenizer, holdout_data)
print("Updated accuracy:", updated_accuracy)


Initial accuracy: 0.6632124352331606
Enter text: Missed an important deadline
Predicted emotion: sadness
Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): 1
New predicted emotion: sadness
Do you have more data to provide? (yes/no): yes
Enter text: Are you kidding me right now
Predicted emotion: sadness
Enter correct label (happiness:0, sadness:1, anger:2, neutral:3): 2
New predicted emotion: neutral
Do you have more data to provide? (yes/no): Are you kidding me right now
Updated accuracy: 0.6632124352331606
